In [1]:
# Test LLM wrapper
from src.llm.OpenAIModel import OpenAIModel
model_config = {"model" : "gpt-4o-mini",
        "temperature" : 0,
        "max_tokens" : None,
        "timeout" : None,
        "max_retries" : 2
        }

model = OpenAIModel(name="open_ai_test", config=model_config)

model.invoke("This a test prompt, say hi")

'Hi! How can I assist you today?'

In [2]:
from src.test.Metric import LLMMetric, Rouge1Metric

random_llm_metric_prompt = "You are a random number generator, you generate number from 1 to 100, you only return said number and nothing more, please return a random number:"
random_llm_metric = LLMMetric("random_llm_metric", prompt=random_llm_metric_prompt, llm=model)

random_llm_metric.eval("predicted", "target", "instruction")
rouge1_metric = Rouge1Metric()
rouge1_metric.eval("predicted", "predicted", "instruction")


1.0

In [3]:
from src.test.TestCase import TestCase

test_case1 = TestCase(input_vars={}, expected_output="expected output", generated_output="generated output")
test_case2 = TestCase(input_vars={}, expected_output="same", generated_output="same")

In [6]:
from src.test.Evaluator import WeightedSumEvaluator

custom_eval = WeightedSumEvaluator(metrics=[random_llm_metric, rouge1_metric], weights={rouge1_metric: 1, "random_llm_metric": 100})

In [7]:
custom_eval.evaluate(test_case=test_case2, instruction="instruction")

0.5742574257425742